In [1]:
import copy
import os

import pandas as pd
import geopandas as  gpd
import psycopg2
import ipyleaflet as leaflet
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual


In [2]:
def connect():
    return psycopg2.connect(
        host="localhost",
        database="berlin_accidents",
        user="postgres",
        password=os.getenv("DB_PASS")
    )
conn = connect()

In [3]:
def dbscan_cmd(eps=0.0015, minPoints=10):
    dbscan = f"""
    SELECT st_setsrid(st_convexhull(st_collect(pt_geom)),4326) as geom, cluster_id
    FROM (
        SELECT objectid, pt_geom, ST_ClusterDBSCAN(pt_geom, eps := {eps}, minPoints := {minPoints})
            OVER() AS cluster_id FROM accidents
    ) sub
    where cluster_id is not null
    group by cluster_id
    """
    return dbscan


def within_cmd(distance=0.0015, minPoints=2):
    within = f"""
    SELECT st_setsrid(st_convexhull(sub.gc), 4326) as geom
        FROM (
            SELECT unnest(ST_ClusterWithin(pt_geom, {distance})) as gc
            FROM accidents
        ) as sub
        WHERE ST_NumGeometries(sub.gc) >= {minPoints}
    """
    return within
    

In [4]:
def plot_map(data):
    m = leaflet.Map(center=(52.5200, 13.4050), zoom=12, basemap=leaflet.basemaps.OpenStreetMap.BlackAndWhite)
    m.add_control(leaflet.LayersControl())
    geo_data = leaflet.GeoData(geo_dataframe = data)
    m.add_layer(geo_data)
    return copy.copy(m)

In [14]:
def _interact_dbscan_cluster(eps, minPoints):
    eps = eps/1000
    print(eps, minPoints)
    d = gpd.read_postgis(dbscan_cmd(eps, minPoints), conn, crs=4326)
    return plot_map(d)


def _interact_within_cluster(eps, minPoints):
    eps = eps/1000
    print(eps, minPoints)
    d = gpd.read_postgis(within_cmd(eps, minPoints), conn, crs=4326)
    return plot_map(d)


In [12]:
dbscan_interact = interact_manual(
    _interact_dbscan_cluster,
    eps=widgets.FloatSlider(min=0.1, max=3, step=0.05, value=2),
    minPoints=widgets.IntSlider(min=1, max=100, step=3, value=5))

within_interact = interact_manual(
    _interact_within_cluster,
    eps=widgets.FloatSlider(min=0.1, max=3, step=0.05, value=2),
    minPoints=widgets.IntSlider(min=1, max=100, step=1, value=5))



interactive(children=(FloatSlider(value=2.0, description='eps', max=3.0, min=0.1, step=0.05), IntSlider(value=…

interactive(children=(FloatSlider(value=2.0, description='eps', max=3.0, min=0.1, step=0.05), IntSlider(value=…

In [13]:
from ipywidgets import Layout, Button, Box

box_layout = Layout(
    display='flex',
    flex_flow='row',
    border='solid',
    align_items='stretch',
    justify_content='space-around',
    width='100%'
)

items = [dbscan_interact.widget, within_interact.widget]

box = Box(children=items, layout=box_layout)
box

Box(children=(interactive(children=(FloatSlider(value=2.0, description='eps', max=3.0, min=0.1, step=0.05), In…